# AutoGluon Image Classification Finetuning

**Import the basic packages**

In [1]:
import autogluon as ag
from gluoncv.model_zoo import get_model
from autogluon import image_classification as task

**Create dataset**

In [2]:
dataset = task.Dataset(name='apparel',
                       train_path='datasets/apparel/train/',
                       val_path='datasets/apparel/val/',
                       num_classes=18)
print(dataset)

AutoGluon Dataset: 
 name = apparel


**Required resources per trial**

In [3]:
resources_per_trial = {
    'max_num_gpus': 4,
    'max_num_cpus': 32,
    'max_training_epochs': 80
}

**Start Fitting**

In [4]:
results = task.fit(dataset, resources_per_trial=resources_per_trial)

In [5]:
print('%.2f acc' % (results.metric * 100))
print(results.config)

67.62 acc
{'loss': 'SoftmaxCrossEntropyLoss', 'metric': 'Accuracy', 'model': 'resnet50_v1b', 'optimizer': 'sgd', 'lr': 0.004933784345432528, 'momentum': 0.8669032341970383, 'norm_layer': 'BatchNorm', 'pretrained': True, 'pretrained_base': True, 'wd': 0.00019788152102079938}


## Customized Hyperparameters for Search
search learning rate and weight decay

In [6]:
opt = ag.optims.SGD(lr=ag.Log('lr', 0.001, 0.01),
                    wd=ag.Log('wd', 1e-5, 1e-3))
optimizers = ag.Optimizers([opt])
print(optimizers)

AutoGluon Optimizers ['sgd'] with Configuration space object:
  Hyperparameters:
    lr, Type: UniformFloat, Range: [0.001, 0.01], Default: 0.0031622777, on log-scale
    optimizer, Type: Categorical, Choices: {sgd}, Default: sgd
    wd, Type: UniformFloat, Range: [1e-05, 0.001], Default: 0.0001, on log-scale
  Conditions:
    lr | optimizer in {'sgd'}
    wd | optimizer in {'sgd'}



In [7]:
stop_criterion = {
    'time_limits': 1*60*60,
    'max_metric': 1.0,
    'max_trial_count': 20,
}

results = task.fit(dataset, optimizers=optimizers,
                   resources_per_trial=resources_per_trial,
                   stop_criterion=stop_criterion,
                   trial_scheduler='hyperband')

In [8]:
print('%.2f acc' % (results.metric * 100))
print(results.config)

68.14 acc
{'loss': 'SoftmaxCrossEntropyLoss', 'metric': 'Accuracy', 'model': 'resnet18_v1b', 'optimizer': 'sgd', 'lr': 0.00676362356656999, 'norm_layer': 'BatchNorm', 'pretrained': True, 'pretrained_base': False, 'wd': 1.1926031323492988e-05}
